### 필요 라이브러리 설치

In [ ]:
!pip install -qU langchain-openai langchain-core
!pip install langchain-community
!pip install gradio>=5.0 numpy

In [5]:
# 각 포트에서 실행 중인 프로세스를 확인
!lsof -i :7865
!lsof -i :7866
!lsof -i :7867

## 그라디오 창 구성하기
고정된 답변을 반환하는 챗봇 형식

In [ ]:
# gradio 라이브러리를 gr 라는 이름으로 import
import gradio as gr

# gradio 내부에서 사용할 함수
def update(name):
    return f"Welcome to Gradio, {name}!"

with gr.Blocks() as iface:
    gr.Markdown("Hello, World! I’m yelling in gradio!")
    with gr.Row():
        inp = gr.Textbox(placeholder="이름이 무엇인가요?")
        out = gr.Textbox()
    btn = gr.Button("제출")

    # 버튼에 이벤트 리스너를 추가
    # 버튼 클릭시 update함수를 호출 -> inp에 입력된 문자열을 파라미터로 전송. 함수의 반환값은 out에 출력
    btn.click(fn=update, inputs=inp, outputs=out)

iface.launch()

In [ ]:
iface.close()

답변 내용 구체화 가능

In [ ]:
# 챗봇에 채팅이 입력되면 이 함수를 호출
# message는 유저의 채팅 메시지, history는 채팅 기록, additional_input_info는 additional_inputs안 블록의 정보
def response(message, history, additional_input_info):
    # additional_input_info의 텍스트를 챗봇의 대답 뒤에 추가
    return "챗봇을 미완성하였습니다 " + additional_input_info

iface = gr.ChatInterface(
    fn=response,
    textbox=gr.Textbox(placeholder="나만의 챗봇과 대화해봐요!!", container=False, scale=1), # 버튼의 크기
    title="나만의 챗봇",
    description="나만의 챗봇으로 Customizing하여 대화하기!",
    theme="soft",
    examples=[["안뇽"], ["요즘 덥다 ㅠㅠ"], ["점심메뉴 추천바람, 짜장 짬뽕 택 1"]],
    retry_btn="다시 실행하기 ↩",
    undo_btn="이전 대화 삭제 ❌",
    clear_btn="전체 대화 삭제 💫",
    additional_inputs=[
        gr.Textbox("!!!", label="e.g. 끝말잇기")
    ]    
)
iface.launch()

In [ ]:
iface.close()

언어 모델을 실제로 불러와서 답변 호출하기

In [13]:
import getpass
import os
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API key를 입력하세요.')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-4o-mini')

def response(message, history, additional_input_info):
        history_langchain_format = []
        # additional_input_info로 받은 시스템 프롬프트를 랭체인에게 전달할 메시지에 포함시킨다.
        history_langchain_format.append(SystemMessage(content= additional_input_info))
        for human, ai in history:
                history_langchain_format.append(HumanMessage(content=human))
                history_langchain_format.append(AIMessage(content=ai))
        history_langchain_format.append(HumanMessage(content=message))
        gpt_response = llm.invoke(history_langchain_format)
        return gpt_response.content

system_prompt = """
당신은 8세 아동이고, 아이처럼 말하는 것에 능숙하다.
사용자의 질문에 대해 아이처럼 대화를 하여라.
"""

iface = gr.ChatInterface(
        fn=response,
        textbox=gr.Textbox(placeholder="나만의 챗봇과 대화해봐요!!", container=False, scale=5),
        chatbot=gr.Chatbot(height=300), # 채팅창 높이 조절
        title="나만의 챗봇",
        description="나만의 챗봇으로 Customizing하여 대화하기!",
        theme="soft",
        examples=[["안뇽"], ["요즘 덥다 ㅠㅠ"], ["점심메뉴 추천바람, 짜장 짬뽕 택 1"]],
        retry_btn="다시 작성하기 ↩",
        undo_btn="이전 대화 삭제 ❌",
        clear_btn="전체 대화 삭제 💫",
        additional_inputs=[
            gr.Textbox(system_prompt, label="System Prompt를 입력해주세요", placeholder="8세 아동이고 아이처럼 말하는 것에 능숙하다.")
        ]
)
iface.launch()

In [ ]:
iface.close()

## 프롬프트 엔지니어링
이론 수업에서 배운 내용을 복습해 봅시다.
### 1. 명료하게 지시하기
- 대통령이 누구야? vs 2021년 멕시코의 대통령은 누구야?
- 피보나치 수열 계산하는 코드 작성해줘 vs 피보나치 수열을 효율적으로 개선하는 파이썬 함수를 작성해줘. 그리고 각 코드가 무슨 기능을 하고 왜 쓰여야 하는지 주석을 달아줘.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-4o-mini')

def response(message, history, additional_input_info):
    history_langchain_format = []
    # additional_input_info로 받은 시스템 프롬프트를 랭체인에게 전달할 메시지에 포함시킨다.
    history_langchain_format.append(SystemMessage(content=additional_input_info))
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = llm.invoke(history_langchain_format)
    return gpt_response.content

system_prompt = """인공지능 모델로서 유저의 질문이나 요청에 적절히 응답하세요.
유저가 사용하는 언어에 맞춰서 한국어 또는 영어로 답변해야 합니다.
또한 유저가 사용하는 말투와 톤에 맞춰서 적절히 응답하도록 하세요."""

iface = gr.ChatInterface(
    fn=response,
    textbox=gr.Textbox(placeholder="나만의 챗봇과 대화해봐요!!", container=False, scale=5),
    chatbot=gr.Chatbot(height=300),  # 채팅창 높이 조절
    title="나만의 챗봇",
    description="나만의 챗봇으로 Customizing하여 대화하기!",
    theme="soft",
    examples=[["안녕, 반가워!"], ["요즘 날씨 덥다 ㅠㅠ,를 영어로 번역해줘"], ["점심메뉴 추천바람, 짜장 짬뽕 택 1"]],
    retry_btn="다시 작성하기 ↩",
    undo_btn="이전 대화 삭제 ❌",
    clear_btn="전체 대화 삭제 💫",
    additional_inputs=[
        gr.Textbox(system_prompt, label="System Prompt를 입력해주세요", placeholder="어린 아이가 충분히 이해할 수 있도록 친절하게 답변해주세요.")
    ]
)
iface.launch()

In [ ]:
iface.close()

### 2. 참고 가능한 텍스트를 제공하기
- 아이폰 15 가격 얼마야?
- 텍스트 파일 불러오기

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-4o-mini')

def response(message, history, additional_input_info):
        history_langchain_format = []
        # additional_input_info로 받은 시스템 프롬프트를 랭체인에게 전달할 메시지에 포함시킨다.
        history_langchain_format.append(SystemMessage(content= additional_input_info))
        for human, ai in history:
                history_langchain_format.append(HumanMessage(content=human))
                history_langchain_format.append(AIMessage(content=ai))
        history_langchain_format.append(HumanMessage(content=message))
        gpt_response = llm.invoke(history_langchain_format)
        return gpt_response.content

system_prompt = """인공지능 모델로서 유저의 질문이나 요청에 적절히 응답하세요.
유저가 사용하는 언어에 맞춰서 한국어 또는 영어로 답변해야 합니다.
또한 유저가 사용하는 말투와 톤에 맞춰서 적절히 응답하도록 하세요.
당신의 지식으로 답변하기 어려운 질문에 대해서는 모른다고 답변해야 합니다."""

iface = gr.ChatInterface(
        fn=response,
        textbox=gr.Textbox(placeholder="나만의 챗봇과 대화해봐요!!", container=False, scale=5),
        chatbot=gr.Chatbot(height=300), # 채팅창 높이 조절
        title="나만의 챗봇",
        description="나만의 챗봇으로 Customizing하여 대화하기!",
        theme="soft",
        examples=[["안녕, 반가워!"], ["요즘 날씨 덥다 ㅠㅠ,를 영어로 번역해줘"], ["점심메뉴 추천바람, 짜장 짬뽕 택 1"]],
        retry_btn="다시 작성하기 ↩",
        undo_btn="이전 대화 삭제 ❌",
        clear_btn="전체 대화 삭제 💫",
        additional_inputs=[
            gr.Textbox(system_prompt, label="System Prompt를 입력해주세요", placeholder="어린 아이가 충분히 이해할 수 있도록 친절하게 답변해주세요.")
        ]
)
iface.launch(share=True)

In [ ]:
# 텍스트 파일 불러오기
with open('iphone15_info.txt', 'r') as file:
    text = file.read()
print(text)

In [ ]:
iface.close()

### 3. 복잡한 작업은 여러 개의 작은 작업으로 쪼개기

#다른걸로 해야할듯

- 공 튀기기 게임을 만드는 코드를 작성해줘.
- 공 튀기기 게임을 만드는 코드를 작성해줘. 아래 내용을 따라 코드를 작성하면 돼.

 	1.	기본 구조 설정
“Python으로 공 튀기기 게임을 만들기 위해 Pygame을 설치하고 기본 게임 창을 만드는 코드를 작성해줘.”
	2.	공 객체 만들기
“Pygame을 사용하여 화면에 표시될 공 객체를 정의하는 코드를 작성해줘. 공은 원형으로 그리고, 초기 위치와 속도를 설정해줘.”
	3.	공의 움직임 구현
“Pygame을 사용하여 공이 화면에서 움직이도록 하는 코드를 작성해줘. 공이 벽에 부딪히면 튕기도록 해줘.”
	4.	게임 루프 설정
“게임이 지속적으로 실행될 수 있도록 게임 루프를 설정하고, 공의 위치를 업데이트하며 화면을 다시 그리는 코드를 작성해줘.”
	5.	사용자 입력 처리
“사용자가 공의 위치를 조작할 수 있도록 방향키 입력에 영향을 받을 수 있도록 해줘.”
	6.	완성된 게임 실행
“모든 코드를 합쳐서 완성된 공 튀기기 게임을 실행할 수 있도록 해줘.”


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-4o-mini')

def response(message, history, additional_input_info):
        history_langchain_format = []
        # additional_input_info로 받은 시스템 프롬프트를 랭체인에게 전달할 메시지에 포함시킨다.
        history_langchain_format.append(SystemMessage(content= additional_input_info))
        for human, ai in history:
                history_langchain_format.append(HumanMessage(content=human))
                history_langchain_format.append(AIMessage(content=ai))
        history_langchain_format.append(HumanMessage(content=message))
        gpt_response = llm.invoke(history_langchain_format)
        return gpt_response.content

system_prompt = """인공지능 모델로서 유저의 질문이나 요청에 적절히 응답하세요.
유저가 사용하는 언어에 맞춰서 한국어 또는 영어로 답변해야 합니다.
또한 유저가 사용하는 말투와 톤에 맞춰서 적절히 응답하도록 하세요.
유저의 요청을 최대한 잘 들어줄 수 있도록 합니다."""

iface = gr.ChatInterface(
        fn=response,
        textbox=gr.Textbox(placeholder="나만의 챗봇과 대화해봐요!!", container=False, scale=5),
        chatbot=gr.Chatbot(height=300), # 채팅창 높이 조절
        title="나만의 챗봇",
        description="나만의 챗봇으로 Customizing하여 대화하기!",
        theme="soft",
        examples=[["안녕, 반가워!"], ["요즘 날씨 덥다 ㅠㅠ,를 영어로 번역해줘"], ["점심메뉴 추천바람, 짜장 짬뽕 택 1"]],
        retry_btn="다시 작성하기 ↩",
        undo_btn="이전 대화 삭제 ❌",
        clear_btn="전체 대화 삭제 💫",
        additional_inputs=[
            gr.Textbox(system_prompt, label="System Prompt를 입력해주세요", placeholder="어린 아이가 충분히 이해할 수 있도록 친절하게 답변해주세요.")
        ]
)
iface.launch()

In [ ]:
iface.close()

간단한 명령으로 생성된 결과


In [ ]:
import pygame
import sys

# Pygame 초기화
pygame.init()

# 게임 창 설정
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('공 튀기기 게임')

# 색상 정의
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

class Ball:
    def __init__(self, x, y, radius, color):
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color
        self.speed_x = 5  # x 방향 속도
        self.speed_y = 5  # y 방향 속도

    def draw(self, surface):
        pygame.draw.circle(surface, self.color, (self.x, self.y), self.radius)

    def move(self):
        self.x += self.speed_x
        self.y += self.speed_y

        # 화면 경계 검사
        if self.x - self.radius < 0 or self.x + self.radius > WIDTH:
            self.speed_x = -self.speed_x  # X 방향으로 튕김
        if self.y - self.radius < 0 or self.y + self.radius > HEIGHT:
            self.speed_y = -self.speed_y  # Y 방향으로 튕김

# 공 객체 생성
ball = Ball(WIDTH // 2, HEIGHT // 2, 20, WHITE)

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        ball.speed_x = -5
    elif keys[pygame.K_RIGHT]:
        ball.speed_x = 5
    else:
        ball.speed_x = 0  # 방향키를 누르지 않으면 정지
    if keys[pygame.K_UP]:
        ball.speed_y = -5
    elif keys[pygame.K_DOWN]:
        ball.speed_y = 5
    else:
        ball.speed_y = 0  # 방향키를 누르지 않으면 정지

    # 공의 이동 및 그리기
    ball.move()

    # 화면 채우기
    screen.fill(BLACK)
    
    # 공 그리기
    ball.draw(screen)

    # 화면 업데이트
    pygame.display.flip()

    # FPS 설정
    pygame.time.Clock().tick(60)

# Pygame 종료
pygame.quit()
sys.exit()

복잡한 명령으로 생성된 결과

In [ ]:
import pygame
pygame.init()

# 게임 화면 크기 설정
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("공 튀기기 게임")

# 공 객체 설정
class Ball:
    def __init__(self, x, y, radius, color, speed_x, speed_y):
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color
        self.speed_x = speed_x
        self.speed_y = speed_y

    def draw(self):
        pygame.draw.circle(screen, self.color, (self.x, self.y), self.radius)

ball = Ball(screen_width // 2, screen_height // 2, 20, (255, 0, 0), 5, 5)

# 공의 움직임 구현
def move_ball():
    ball.x += ball.speed_x
    ball.y += ball.speed_y

    if ball.x <= ball.radius or ball.x >= screen_width - ball.radius:
        ball.speed_x *= -1
    if ball.y <= ball.radius or ball.y >= screen_height - ball.radius:
        ball.speed_y *= -1

# 게임 루프
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False

    screen.fill((255, 255, 255))
    
    move_ball()
    ball.draw()

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        ball.speed_x = -5
    elif keys[pygame.K_RIGHT]:
        ball.speed_x = 5
    if keys[pygame.K_UP]:
        ball.speed_y = -5
    elif keys[pygame.K_DOWN]:
        ball.speed_y = 5

    pygame.display.update()

pygame.quit()

### 4. 페르소나 설정
- 당신은 인공지능 모델로서 어린 초등학생의 질문을 받게 됩니다. 초등학생의 질문에 대해 최대한 자세하고 풍부한 설명이 필요합니다. 또한 친절하고 친근한 말투를 유지할 수 있도록 하세요. 가능한 경우 이모지를 추가하여 친근함을 더하도록 합니다.
- 당신은 인공지능 전문가로서 대학생의 질문을 받게 됩니다. 데이터 사이언스와 관련한 질문에 대해 전문적인 내용을 바탕으로 답변합니다. 이때 전문적인 느낌을 줄 수 있는 말투를 유지하도록 하세요.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-4o-mini')

def response(message, history, additional_input_info):
        history_langchain_format = []
        # additional_input_info로 받은 시스템 프롬프트를 랭체인에게 전달할 메시지에 포함시킨다.
        history_langchain_format.append(SystemMessage(content= additional_input_info))
        for human, ai in history:
                history_langchain_format.append(HumanMessage(content=human))
                history_langchain_format.append(AIMessage(content=ai))
        history_langchain_format.append(HumanMessage(content=message))
        gpt_response = llm.invoke(history_langchain_format)
        return gpt_response.content

system_prompt = """당신은 인공지능 모델로서 어린 초등학생의 질문을 받게 됩니다. 초등학생의 질문에 대해 최대한 자세하고 풍부한 설명이 필요합니다. 또한 친절하고 친근한 말투를 유지할 수 있도록 하세요. 가능한 경우 이모지를 추가하여 친근함을 더하도록 합니다."""

iface = gr.ChatInterface(
        fn=response,
        textbox=gr.Textbox(placeholder="나만의 챗봇과 대화해봐요!!", container=False, scale=5),
        chatbot=gr.Chatbot(height=300), # 채팅창 높이 조절
        title="나만의 챗봇",
        description="나만의 챗봇으로 Customizing하여 대화하기!",
        theme="soft",
        examples=[["안녕, 반가워!"], ["요즘 날씨 덥다 ㅠㅠ,를 영어로 번역해줘"], ["피보나치 수열이 뭔지 알아? 알면 파이썬으로 코딩해줘"]],
        retry_btn="다시 작성하기 ↩",
        undo_btn="이전 대화 삭제 ❌",
        clear_btn="전체 대화 삭제 💫",
        additional_inputs=[
            gr.Textbox(system_prompt, label="System Prompt를 입력해주세요", placeholder="어린 아이가 충분히 이해할 수 있도록 친절하게 답변해주세요.")
        ]
)
iface.launch()

In [ ]:
iface.close()

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-4o-mini')

def response(message, history, additional_input_info):
        history_langchain_format = []
        # additional_input_info로 받은 시스템 프롬프트를 랭체인에게 전달할 메시지에 포함시킨다.
        history_langchain_format.append(SystemMessage(content= additional_input_info))
        for human, ai in history:
                history_langchain_format.append(HumanMessage(content=human))
                history_langchain_format.append(AIMessage(content=ai))
        history_langchain_format.append(HumanMessage(content=message))
        gpt_response = llm.invoke(history_langchain_format)
        return gpt_response.content

system_prompt = """당신은 인공지능 전문가로서 대학생의 질문을 받게 됩니다. 데이터 사이언스와 관련한 질문에 대해 전문적인 내용을 바탕으로 답변합니다. 이때 전문적인 느낌을 줄 수 있는 말투를 유지하도록 하세요."""

iface = gr.ChatInterface(
        fn=response,
        textbox=gr.Textbox(placeholder="나만의 챗봇과 대화해봐요!!", container=False, scale=5),
        chatbot=gr.Chatbot(height=300), # 채팅창 높이 조절
        title="나만의 챗봇",
        description="나만의 챗봇으로 Customizing하여 대화하기!",
        theme="soft",
        examples=[["안녕, 반가워!"], ["요즘 날씨 덥다 ㅠㅠ,를 영어로 번역해줘"], ["피보나치 수열이 뭔지 알아? 알면 파이썬으로 코딩해줘"]],
        retry_btn="다시 작성하기 ↩",
        undo_btn="이전 대화 삭제 ❌",
        clear_btn="전체 대화 삭제 💫",
        additional_inputs=[
            gr.Textbox(system_prompt, label="System Prompt를 입력해주세요", placeholder="어린 아이가 충분히 이해할 수 있도록 친절하게 답변해주세요.")
        ]
)
iface.launch()

In [ ]:
iface.close()

## 나만의 자소서 챗봇 만들기!!
마지막 실습 과제는 자소서를 작성해주는 챗봇을 직접 만들어보는 것입니다.

본인만의 자소서를 챗봇으로 생성하여 결과물을 제출해 보세요!

만약 답변이 마음에 들지 않는다면 이를 어떻게 수정해달라고 요청할지 고민해보도록 합니다 🤔

단, 자소서에는 아래 항목을 반드시 포함해야 합니다.
- 가장 인상 깊게 수강했던 전공 과목
- 친구/동료들과 겪었던 갈등을 해결했던 경험
- 본인의 건강한 취미 생활

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-4o-mini')

def response(message, history, additional_input_info):
        history_langchain_format = []
        # additional_input_info로 받은 시스템 프롬프트를 랭체인에게 전달할 메시지에 포함시킨다.
        history_langchain_format.append(SystemMessage(content= additional_input_info))
        for human, ai in history:
                history_langchain_format.append(HumanMessage(content=human))
                history_langchain_format.append(AIMessage(content=ai))
        history_langchain_format.append(HumanMessage(content=message))
        gpt_response = llm.invoke(history_langchain_format)
        return gpt_response.content

system_prompt = """""" ## 본인만의 프롬프트를 작성해 보세요

iface = gr.ChatInterface(
        fn=response,
        textbox=gr.Textbox(placeholder="나만의 챗봇과 대화해봐요!!", container=False, scale=5),
        chatbot=gr.Chatbot(height=300), # 채팅창 높이 조절
        title="나만의 챗봇",
        description="나만의 챗봇으로 Customizing하여 대화하기!",
        theme="soft",
        examples=[["안녕, 반가워!"], ["요즘 날씨 덥다 ㅠㅠ,를 영어로 번역해줘"], ["피보나치 수열이 뭔지 알아? 알면 파이썬으로 코딩해줘"]],
        retry_btn="다시 작성하기 ↩",
        undo_btn="이전 대화 삭제 ❌",
        clear_btn="전체 대화 삭제 💫",
        additional_inputs=[
            gr.Textbox(system_prompt, label="System Prompt를 입력해주세요", placeholder="질문에 친절하게 답변하는 챗봇")
        ]
)
iface.launch()

In [ ]:
iface.close()